In [1]:
from transformers import LlamaForSequenceClassification
import torch
from torch.nn.utils import clip_grad_value_
import torch.optim as optim
import numpy as np
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
MAX_LEN = 1200

##### change model path and data used to train!

#### 1. Fine-Tune Value Function

In [2]:
version = "1"
Model_Path = f'../Model/PRM_LORA_merge{version}_code'
head_path = f'../Model/model_score{version}_code.pth'
next_version = str(int(version) + 1)

In [3]:
import pickle
import re
clean_text = lambda x:re.sub(r"(<math>|<\/math>|<cmath>|<\/cmath>|\\begin\{align\*\}|\\end\{align\*\})", "", x)

# RL data
with open(f"../llmOutputs/PRM/data_V1_code{version}.pickle", "rb") as f:
    data_V = pickle.load(f)
with open(f"../llmOutputs/PRM/completed_paths_y_code{version}.pickle", "rb") as f:
    completed_paths_y = pickle.load(f)

data = []
for text,y in data_V:
    data.append([clean_text(text.replace("<｜begin▁of▁sentence｜>User: ","")),y])
for y,score,text,code,prob_i,exit_i in completed_paths_y:
    data.append([clean_text(text.replace("<｜begin▁of▁sentence｜>User: ","")),y])

# SFT data # TODO: remove this later? if loss is too low, e.g. <0.1, overfit or topic classification
with open(f"../Data/ai-mathematical-olympiad-prize/10prob.pickle", "rb") as f:
    outs = pickle.load(f)
with open(f"../Data/AMC/aime_final.pickle", "rb") as f:
    outs2 = pickle.load(f)
for q,s in outs:
    if np.random.rand()<0.5:
        data.append([clean_text(q+s),1])
for q,s in outs2:
    if np.random.rand()<0.25:
        data.append([clean_text(q+s),1])
import random
random.shuffle(data)
texts,ys = zip(*data)

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-math-7b-rl")
texts = tokenizer.batch_encode_plus(texts,return_attention_mask=False,add_special_tokens=True,\
                                    truncation=True, max_length=MAX_LEN)['input_ids']

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def from_gen(texts,ys):
    data = list(zip(texts,ys))
    random.shuffle(data)
    for text,y in data:
        text = torch.tensor(text,device='cuda')[None]
        y = torch.tensor([y],device='cuda',dtype=torch.float32)
        yield text,y

In [6]:
epochs = 1
accumulation_steps = 64
verbose = 1024
lr = 6e-5
clip = 6e-3

In [7]:
from transformers import LlamaForSequenceClassification,BitsAndBytesConfig,AutoConfig
import torch
from peft import (
    get_peft_model,
    PeftType,
    LoraConfig)

In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = LlamaForSequenceClassification.from_pretrained(Model_Path,\
                                                       num_labels=1,\
                                                       device_map="auto",
                                                       torch_dtype="auto",
                                                       quantization_config=quantization_config,
                                                       attn_implementation="flash_attention_2"
                                                       )
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at ../Model/PRM_LORA_merge1_code and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
peft_config = LoraConfig(r=8, # low rank 
                         lora_alpha = 16, # see below 
                         lora_dropout = 0.1, 
                         bias="none",#'none', 'all' or 'lora_only' 
                         target_modules = [ "q_proj", 
                                            "k_proj", 
                                            "v_proj", 
                                            "o_proj", 
                                            "gate_proj", 
                                            "up_proj", 
                                            "down_proj" 
                                        ] 
                        )
base_model = get_peft_model(model.model, peft_config)
base_model.gradient_checkpointing_enable()
# model.config.pad_token_id = tokenizer.pad_token_id
base_model.print_trainable_parameters()
model.score = model.score.float()
model.score.load_state_dict(torch.load(head_path))
model.score.weight.requires_grad_(True);

trainable params: 18,739,200 || all params: 6,509,674,496 || trainable%: 0.2879


FT head

In [10]:
base_params = [param for param in base_model.parameters() if param.requires_grad]
trainable_params = list(model.score.parameters())
                    # list(topic_model.parameters())
optimizer = torch.optim.Adam(trainable_params,lr = lr)

In [11]:
# %debug
loss_fn = torch.nn.BCEWithLogitsLoss()
train_loss = 0
count_loss = 0

for epoch in range(epochs):
    for i,(text,y) in enumerate(from_gen(texts,ys)):
        with torch.no_grad():
            hidden_states = base_model(text)[0][:,-1].float() # 1,d
        logits = model.score(hidden_states)[:,0] # 1,
        loss = loss_fn(logits,y)
        loss.backward()
        train_loss += loss.item()
        count_loss += 1
            
        if (i + 1) % accumulation_steps == 0:
            # clip_grad_value_(trainable_params,clip)
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()

        if (i + 1) % verbose == 0:
            print(f"iter: {i}, \n train loss: {train_loss/count_loss}")
            train_loss = 0
            count_loss = 0
            
        torch.cuda.empty_cache()

iter: 1023, 
 train loss: 1.0186482066653753
iter: 2047, 
 train loss: 0.9184768533968963
iter: 3071, 
 train loss: 0.7912754519243208
iter: 4095, 
 train loss: 0.7327570735851623
iter: 5119, 
 train loss: 0.7035882661257347
iter: 6143, 
 train loss: 0.6486897827771827
iter: 7167, 
 train loss: 0.6083336817973759
iter: 8191, 
 train loss: 0.5384236243626219
iter: 9215, 
 train loss: 0.5246589001690154
iter: 10239, 
 train loss: 0.5247169548310922
iter: 11263, 
 train loss: 0.4997896201966796
iter: 12287, 
 train loss: 0.4533360378263751
iter: 13311, 
 train loss: 0.45762812863540603
iter: 14335, 
 train loss: 0.4415243639596156
iter: 15359, 
 train loss: 0.4619315887612174
iter: 16383, 
 train loss: 0.4164173425597255
iter: 17407, 
 train loss: 0.42625634140131297
iter: 18431, 
 train loss: 0.4295422995419358
iter: 19455, 
 train loss: 0.43823870234336937
iter: 20479, 
 train loss: 0.41908886995952344
iter: 21503, 
 train loss: 0.430398246509867
iter: 22527, 
 train loss: 0.42396796754

FT head and backbone

In [12]:
base_params = [param for param in base_model.parameters() if param.requires_grad]
trainable_params =  base_params + list(model.score.parameters())
                    # list(topic_model.parameters())
optimizer = torch.optim.Adam(trainable_params,lr = lr)

In [13]:
loss_fn = torch.nn.BCEWithLogitsLoss()
train_loss = 0
count_loss = 0

for epoch in range(epochs):
    for i,(text,y) in enumerate(from_gen(texts,ys)):
        hidden_states = base_model(text)[0][:,-1].float() # 1,d
        logits = model.score(hidden_states)[:,0] # 1,
        loss = loss_fn(logits,y)
        loss.backward()
        train_loss += loss.item()
        count_loss += 1
            
        if (i + 1) % accumulation_steps == 0:
            # clip_grad_value_(trainable_params,clip)
            clip_grad_value_(trainable_params,clip)
            optimizer.step()
            optimizer.zero_grad()

        if (i + 1) % verbose == 0:
            print(f"iter: {i}, \n train loss: {train_loss/count_loss}")
            train_loss = 0
            count_loss = 0
            
        torch.cuda.empty_cache()

iter: 1023, 
 train loss: 0.41366537037538365
iter: 2047, 
 train loss: 0.41620476913885796
iter: 3071, 
 train loss: 0.37813808389455517
iter: 4095, 
 train loss: 0.35819730408866235
iter: 5119, 
 train loss: 0.32872925177730394
iter: 6143, 
 train loss: 0.3380288858639915
iter: 7167, 
 train loss: 0.3206686378935615
iter: 8191, 
 train loss: 0.30413909911527526
iter: 9215, 
 train loss: 0.3071033009244388
iter: 10239, 
 train loss: 0.30804539255632335
iter: 11263, 
 train loss: 0.3018386588447015
iter: 12287, 
 train loss: 0.33025867337934756
iter: 13311, 
 train loss: 0.288254887629094
iter: 14335, 
 train loss: 0.29319452295891324
iter: 15359, 
 train loss: 0.26809748683029966
iter: 16383, 
 train loss: 0.2640008098639015
iter: 17407, 
 train loss: 0.26303629281852636
iter: 18431, 
 train loss: 0.25689785744985727
iter: 19455, 
 train loss: 0.30667372214721667
iter: 20479, 
 train loss: 0.26606939207488267
iter: 21503, 
 train loss: 0.2955766957239945
iter: 22527, 
 train loss: 0.2

In [14]:
torch.save(model.score.state_dict(), f'../Model/model_score{next_version}_code.pth')
peft_model_id = f"../Model/PRM_LORA{next_version}_code"
!mkdir peft_model_id
base_model.save_pretrained(peft_model_id)

/home/zhenlan/anaconda3/envs/torch/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ../Model/PRM_LORA_merge1_code - will assume that the vocabulary was not modified.
  warnings.warn(


In [15]:
del model,base_model,texts,hidden_states,logits,loss
import gc
gc.collect()
torch.cuda.empty_cache()
from transformers import LlamaModel
model = LlamaModel.from_pretrained(Model_Path,\
                                    device_map="auto",
                                    torch_dtype="auto",
                                    attn_implementation="flash_attention_2"
                                    )
from peft import PeftModel
peft_model_id = f"../Model/PRM_LORA{next_version}_code"
base_model = PeftModel.from_pretrained(model, peft_model_id)
base_model2 = base_model.merge_and_unload()
# !mkdir '../Model/PRM_LORA_merge3_code'
base_model2.save_pretrained(f'../Model/PRM_LORA_merge{next_version}_code')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
# from transformers import LlamaForSequenceClassification
# model = LlamaForSequenceClassification.from_pretrained(f'../Model/PRM_LORA_merge{next_version}_code',\
#                                                        num_labels=1,\
#                                                        ignore_mismatched_sizes=True,
#                                                        device_map="auto",
#                                                        torch_dtype="auto",
#                                                        attn_implementation="flash_attention_2"
#                                                        )
# model.score.load_state_dict(torch.load(f'../Model/model_score{next_version}_code.pth'))
# loss_fn = torch.nn.BCEWithLogitsLoss()
# train_loss = 0
# count_loss = 0

# for epoch in range(epochs):
#     for i,(text,y) in enumerate(from_gen(texts,ys)):
#         with torch.no_grad():
#             hidden_states = model.model(text)[0][:,-1] # 1,d
#             logits = model.score(hidden_states)[:,0] # 1,
#             loss = loss_fn(logits,y)
#         train_loss += loss.item()
#         count_loss += 1

#         if (i + 1) % verbose == 0:
#             print(f"iter: {i}, \n train loss: {train_loss/count_loss}")
#             train_loss = 0
#             count_loss = 0
            
#         torch.cuda.empty_cache()